In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from pathlib import Path
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
import sqlite3
import sklearn as skl
import tensorflow as tf
from sklearn.metrics.pairwise import cosine_similarity
from scipy import sparse

# Clean Up

In [2]:
conn = sqlite3.connect('games_data.db')

boardgame_df = pd.read_sql_query("SELECT * FROM boardgame_df;", conn)
boardgame_df

,objectid,name,average,avgweight,boardgamecategory,boardgamemechanic
0,174430,Gloomhaven,8.85292,3.8078,"['Adventure', 'Exploration', 'Fantasy', 'Fight...","['Campaign / Battle Card Driven', 'Cooperative..."
1,161936,Pandemic Legacy Season 1,8.62499,2.8301,"['Environmental', 'Medical']","['Action Points', 'Cooperative Game', 'Hand Ma..."
2,167791,Terraforming Mars,8.42299,3.2313,"['Economic', 'Environmental', 'Industry / Manu...","['Card Drafting', 'End Game Bonuses', 'Hand Ma..."
3,182028,Through the Ages A New Story of Civilization,8.49419,4.3850,"['Card Game', 'Civilization', 'Economic']","['Action Points', 'Auction/Bidding', 'Auction:..."
4,224517,Brass Birmingham,8.62031,3.9122,"['Economic', 'Industry / Manufacturing', 'Tran...","['Hand Management', 'Income', 'Loans', 'Market..."
...,...,...,...,...,...,...
19995,5154,Franchise,6.00000,0.0000,"['Economic', 'Negotiation']","['Simulation', 'Trading']"
19996,5158,Punto y Raya,4.00000,0.0000,['Abstract Strategy'],['Enclosure']
19997,5159,3 Up,6.00000,0.0000,"['Abstract Strategy', 'Childrens Game']",['Pattern Building']
19998,5160,Dino-Checkers,5.77500,0.0000,"['Abstract Strategy', 'Childrens Game']",['Area Majority / Influence']


In [3]:
# Our decoding
decode_lambda = lambda x: bytearray(x, 'utf-8').decode('unicode-escape')
# Applying the decoding to the column, ignore errors.
boardgame_df['name'] = boardgame_df['name'].apply(lambda x: decode_lambda(x))
# Unicode replaced with proper characters
boardgame_df.head(5)

<ipython-input-3-1bb247db571e>:2: DeprecationWarning: invalid escape sequence '\/'
  decode_lambda = lambda x: bytearray(x, 'utf-8').decode('unicode-escape')
<ipython-input-3-1bb247db571e>:2: DeprecationWarning: invalid escape sequence '\T'
  decode_lambda = lambda x: bytearray(x, 'utf-8').decode('unicode-escape')
<ipython-input-3-1bb247db571e>:2: DeprecationWarning: invalid escape sequence '\A'
  decode_lambda = lambda x: bytearray(x, 'utf-8').decode('unicode-escape')
<ipython-input-3-1bb247db571e>:2: DeprecationWarning: invalid escape sequence '\W'
  decode_lambda = lambda x: bytearray(x, 'utf-8').decode('unicode-escape')
<ipython-input-3-1bb247db571e>:2: DeprecationWarning: invalid escape sequence '\O'
  decode_lambda = lambda x: bytearray(x, 'utf-8').decode('unicode-escape')
<ipython-input-3-1bb247db571e>:2: DeprecationWarning: invalid escape sequence '\D'
  decode_lambda = lambda x: bytearray(x, 'utf-8').decode('unicode-escape')
<ipython-input-3-1bb247db571e>:2: DeprecationWarning

,objectid,name,average,avgweight,boardgamecategory,boardgamemechanic
0,174430,Gloomhaven,8.85292,3.8078,"['Adventure', 'Exploration', 'Fantasy', 'Fight...","['Campaign / Battle Card Driven', 'Cooperative..."
1,161936,Pandemic Legacy Season 1,8.62499,2.8301,"['Environmental', 'Medical']","['Action Points', 'Cooperative Game', 'Hand Ma..."
2,167791,Terraforming Mars,8.42299,3.2313,"['Economic', 'Environmental', 'Industry / Manu...","['Card Drafting', 'End Game Bonuses', 'Hand Ma..."
3,182028,Through the Ages A New Story of Civilization,8.49419,4.3850,"['Card Game', 'Civilization', 'Economic']","['Action Points', 'Auction/Bidding', 'Auction:..."
4,224517,Brass Birmingham,8.62031,3.9122,"['Economic', 'Industry / Manufacturing', 'Tran...","['Hand Management', 'Income', 'Loans', 'Market..."


In [4]:
boardgame_df = boardgame_df.rename(columns={'objectid': 'ID', 'name': 'Name', 'average': 'Avg Rating', 'avgweight': 'Complexity', 'boardgamecategory': 'Category', 'boardgamemechanic': 'Mechanic'})

# Other languages are supported by the decoding: Row 70
boardgame_df.head(5)

,ID,Name,Avg Rating,Complexity,Category,Mechanic
0,174430,Gloomhaven,8.85292,3.8078,"['Adventure', 'Exploration', 'Fantasy', 'Fight...","['Campaign / Battle Card Driven', 'Cooperative..."
1,161936,Pandemic Legacy Season 1,8.62499,2.8301,"['Environmental', 'Medical']","['Action Points', 'Cooperative Game', 'Hand Ma..."
2,167791,Terraforming Mars,8.42299,3.2313,"['Economic', 'Environmental', 'Industry / Manu...","['Card Drafting', 'End Game Bonuses', 'Hand Ma..."
3,182028,Through the Ages A New Story of Civilization,8.49419,4.3850,"['Card Game', 'Civilization', 'Economic']","['Action Points', 'Auction/Bidding', 'Auction:..."
4,224517,Brass Birmingham,8.62031,3.9122,"['Economic', 'Industry / Manufacturing', 'Tran...","['Hand Management', 'Income', 'Loans', 'Market..."


# Creating ML Dataframe

In [5]:
boardgame_trunc = boardgame_df.drop(["Category","Mechanic"],axis=1)
boardgame_trunc.head()

,ID,Name,Avg Rating,Complexity
0,174430,Gloomhaven,8.85292,3.8078
1,161936,Pandemic Legacy Season 1,8.62499,2.8301
2,167791,Terraforming Mars,8.42299,3.2313
3,182028,Through the Ages A New Story of Civilization,8.49419,4.3850
4,224517,Brass Birmingham,8.62031,3.9122


In [6]:
categories_df = boardgame_df[['ID', 'Category']].copy()
categories_df['Category'] = categories_df['Category'].apply(lambda x: x[1:len(x)-1].split(', '))

mechanics_df = boardgame_df[['ID', 'Mechanic']].copy()
mechanics_df['Mechanic'] = mechanics_df['Mechanic'].apply(lambda x: x[1:len(x)-1].split(', '))

In [7]:
pd.set_option('display.max_rows', 500)

cat_counts = categories_df.explode('Category')
cat_counts.columns.str.replace("'","")
cat_vc = cat_counts.groupby("Category").size().sort_values(ascending=False)

categories_to_replace = cat_vc[cat_vc < 50].index

# # Replace in dataframe
for cats in categories_to_replace:
    cat_counts['Category'] = cat_counts['Category'].replace(cats,"Other")

cat_counts['Category'] = cat_counts['Category'].str.replace("'","")


# # Check to make sure binning was successful
cat_vc_check = cat_counts.groupby("Category").size().sort_values(ascending=False)
cat_vc_check

Category
Card Game                     5686
Wargame                       3500
Fantasy                       2361
Party Game                    1723
Dice                          1712
Fighting                      1565
Abstract Strategy             1541
Childrens Game                1521
Science Fiction               1495
Economic                      1423
Bluffing                      1136
World War II                  1129
Animals                       1107
Humor                         1064
Deduction                     1046
Adventure                     1017
Action / Dexterity             999
Movies / TV / Radio theme      980
Miniatures                     925
Medieval                       888
Exploration                    792
Racing                         709
Ancient                        697
Negotiation                    636
Real-time                      623
Horror                         593
Nautical                       575
Trivia                         572
Other      

In [8]:
cat_counts

,ID,Category
0,174430,Adventure
0,174430,Exploration
0,174430,Fantasy
0,174430,Fighting
0,174430,Miniatures
...,...,...
19997,5159,Abstract Strategy
19997,5159,Childrens Game
19998,5160,Abstract Strategy
19998,5160,Childrens Game


In [9]:
categories_to_replace

Index([''Pike and Shot'', ''American Indian Wars'', ''Game System'',
       ''Korean War'', ''KOSMOS'', ''Schmidt Spiele'',
       ''Ravensburger Spieleverlag GmbH'', ''White Goblin Games'', ''Piatnik'',
       ''Hasbro'',
       ...
       ''Magellan'', ''Manifest Destiny'', ''MasterPieces'',
       ''Cranio Creations'', ''Corax Games'', ''Conclave Editora'',
       ''Coldicutt Games'', ''Mayfair Games'', ''Clementoni'',
       ''u685cu904au5eb5 (Ouyuuan)''],
      dtype='object', name='Category', length=288)

In [10]:
cat_dummies = pd.get_dummies(cat_counts)
cat_final = cat_dummies.groupby("ID").sum()
cat_final

,Category_Abstract Strategy,Category_Action / Dexterity,Category_Adventure,Category_Age of Reason,Category_American Civil War,Category_American Revolutionary War,Category_American West,Category_Ancient,Category_Animals,Category_Arabian,...,Category_Transportation,Category_Travel,Category_Trivia,Category_Video Game Theme,Category_Vietnam War,Category_Wargame,Category_Word Game,Category_World War I,Category_World War II,Category_Zombies
ID,,,,,,,,,,,,,,,,,,,,,
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
292961,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
292962,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
294612,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [11]:
pd.set_option('display.max_rows', 500)

mech_counts = mechanics_df.explode('Mechanic')
mech_vc = mech_counts.groupby("Mechanic").size().sort_values(ascending=False)

mech_types_to_replace = mech_vc[mech_vc < 175].index

# # Replace in dataframe
for mechs in mech_types_to_replace:
    mech_counts['Mechanic'] = mech_counts['Mechanic'].replace(mechs,"Other")

mech_counts['Mechanic'] = mech_counts['Mechanic'].str.replace("'","")


# # Check to make sure binning was successful
mech_vc_check = mech_counts.groupby("Mechanic").size().sort_values(ascending=False)

mech_dummies = pd.get_dummies(mech_counts)
mech_final = mech_dummies.groupby("ID").sum()

In [12]:
dfs_to_merge = [boardgame_trunc, cat_final, mech_final]

boardgame_inter = pd.merge(boardgame_trunc, cat_final, on="ID", how='outer')
boardgame_final = pd.merge(boardgame_inter, mech_final, on="ID", how='outer')

In [13]:
boardgame_final.columns = boardgame_final.columns.str.replace("Category_", "")
boardgame_final.columns = boardgame_final.columns.str.replace("Mechanic_", "")
boardgame_final.columns = boardgame_final.columns.str.replace("  ", " ")

In [14]:
boardgame_final

,ID,Name,Avg Rating,Complexity,Abstract Strategy,Action / Dexterity,Adventure,Age of Reason,American Civil War,American Revolutionary War,...,Storytelling,Take That,Team-Based Game,Tile Placement,Trading,Trick-taking,Variable Phase Order,Variable Player Powers,Voting,Worker Placement
0,174430,Gloomhaven,8.85292,3.8078,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,161936,Pandemic Legacy Season 1,8.62499,2.8301,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,167791,Terraforming Mars,8.42299,3.2313,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
3,182028,Through the Ages A New Story of Civilization,8.49419,4.3850,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,224517,Brass Birmingham,8.62031,3.9122,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19995,5154,Franchise,6.00000,0.0000,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
19996,5158,Punto y Raya,4.00000,0.0000,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
19997,5159,3 Up,6.00000,0.0000,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
19998,5160,Dino-Checkers,5.77500,0.0000,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


# Collaborative Filtering

In [15]:
data = boardgame_final.drop(['Avg Rating', 'Complexity'], axis=1)

# Create a new dataframe without the user ids.
data_items = data.drop('ID', axis=1)
data_items = data_items.set_index('Name')
data_items.head()

,Abstract Strategy,Action / Dexterity,Adventure,Age of Reason,American Civil War,American Revolutionary War,American West,Ancient,Animals,Arabian,...,Storytelling,Take That,Team-Based Game,Tile Placement,Trading,Trick-taking,Variable Phase Order,Variable Player Powers,Voting,Worker Placement
Name,,,,,,,,,,,,,,,,,,,,,
Gloomhaven,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Pandemic Legacy Season 1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Terraforming Mars,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
Through the Ages A New Story of Civilization,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Brass Birmingham,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [16]:
data_items = pd.DataFrame.transpose(data_items)
data_items.head()

Name,Gloomhaven,Pandemic Legacy Season 1,Terraforming Mars,Through the Ages A New Story of Civilization,Brass Birmingham,Twilight Imperium (Fourth Edition),Twilight Struggle,Star Wars Rebellion,Gaia Project,Scythe,...,Triani,Bedlam,Voyage of the Dawn Treader,The Incredible Crash Dummies Crash Test Game,Fascination Checkers,Franchise,Punto y Raya,3 Up,Dino-Checkers,Obsession
Abstract Strategy,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,1,1,1,1
Action / Dexterity,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Adventure,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Age of Reason,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
American Civil War,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [17]:
data_items.reset_index(level=0, inplace=True)
data_items.head()

Name,index,Gloomhaven,Pandemic Legacy Season 1,Terraforming Mars,Through the Ages A New Story of Civilization,Brass Birmingham,Twilight Imperium (Fourth Edition),Twilight Struggle,Star Wars Rebellion,Gaia Project,...,Triani,Bedlam,Voyage of the Dawn Treader,The Incredible Crash Dummies Crash Test Game,Fascination Checkers,Franchise,Punto y Raya,3 Up,Dino-Checkers,Obsession
0,Abstract Strategy,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,1,1,1,1
1,Action / Dexterity,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Adventure,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Age of Reason,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,American Civil War,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [18]:
data_items = data_items.rename(columns={'index': 'Name'})
data_items.columns.names = ['']

data = data_items

data_items = data_items.drop('Name', axis=1)
data_items.head()

,Gloomhaven,Pandemic Legacy Season 1,Terraforming Mars,Through the Ages A New Story of Civilization,Brass Birmingham,Twilight Imperium (Fourth Edition),Twilight Struggle,Star Wars Rebellion,Gaia Project,Scythe,...,Triani,Bedlam,Voyage of the Dawn Treader,The Incredible Crash Dummies Crash Test Game,Fascination Checkers,Franchise,Punto y Raya,3 Up,Dino-Checkers,Obsession
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,1,1,1,1
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [19]:
data.head()

,Name,Gloomhaven,Pandemic Legacy Season 1,Terraforming Mars,Through the Ages A New Story of Civilization,Brass Birmingham,Twilight Imperium (Fourth Edition),Twilight Struggle,Star Wars Rebellion,Gaia Project,...,Triani,Bedlam,Voyage of the Dawn Treader,The Incredible Crash Dummies Crash Test Game,Fascination Checkers,Franchise,Punto y Raya,3 Up,Dino-Checkers,Obsession
0,Abstract Strategy,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,1,1,1,1
1,Action / Dexterity,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Adventure,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Age of Reason,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,American Civil War,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [20]:
# BG-BG CALCULATIONS

magnitude = np.sqrt(np.square(data_items).sum(axis=1))

data_items = data_items.divide(magnitude, axis='index')

def calculate_similarity(data_items):
    data_sparse = sparse.csr_matrix(data_items)
    similarities = cosine_similarity(data_sparse.transpose())
    sim = pd.DataFrame(data=similarities, index= data_items.columns, columns= data_items.columns)
    return sim

data_matrix = calculate_similarity(data_items)

print(data_matrix.loc['Gloomhaven'].nlargest(11))


Gloomhaven                                            1.000000
Frosthaven                                            1.000000
Star Saga                                             0.874193
Middara  Unintentional Malum – Act 1                  0.815986
Descent  Journeys in the Dark (Second Edition)        0.769629
Dark World                                            0.753396
Shadows of Brimstone  Forbidden Fortress              0.748078
Super Dungeon Explore  Forgotten King                 0.746005
Skull Tales  Full Sail!                               0.743410
Dragon Quest                                          0.739001
Dungeons & Dragons  Wrath of Ashardalon Board Game    0.737950
Name: Gloomhaven, dtype: float64


In [21]:
# CATEGORY-BOARDGAME CALCULATIONS - NO NEIGHBORS

category = 'Adventure' # The id of the user for whom we want to generate recommendations
category_index = data[data.Name == category].index.tolist()[0] # Get the frame index

# Get the boardgames in selected category.
category_boardgames = data_items.iloc[category_index]
category_boardgames = category_boardgames[category_boardgames >0].index.values

# Boardgames for all items as a sparse vector.
user_rating_vector = data_items.iloc[category_index]

# Calculate the score.
score = data_matrix.dot(user_rating_vector).div(data_matrix.sum(axis=1))

# Remove the known boardgames from the recommendation.
score = score.drop(category_boardgames)

# Print the matched boardgames in category and the top 20 recommendations.
print(category_boardgames)
print(score.nlargest(20))

['Gloomhaven' 'War of the Ring (Second Edition)' 'The 7th Continent' ...
 'Dicemaster  Doom Cubes' 'Hauberk' "Don't Feed the Gators"]

Ruin                                 0.009416
Source of the Nile                   0.008989
Savage Worlds  Explorer's Edition    0.008166
Inka                                 0.008102
Frischfleisch                        0.007862
Card Rogue                           0.007451
Exploration                          0.007349
Jenseits von Theben                  0.007342
Vampyre                              0.007313
No Thank You                         0.007071
Fabula                               0.007071
Anasazi                              0.006996
HeroQuest  Barbarian Quest Pack      0.006980
HeroQuest  Elf Quest Pack            0.006980
HeroQuest  Wizards of Morcar         0.006980
Verdict                              0.006884
RPGQuest  Greek Mythology            0.006798
RPGQuest  The Knights Templar        0.006798
HeroQuest  Against the Ogre Horde    

In [22]:
# CATEGORY-BOARDGAME CALCULATIONS - WITH NEIGHBORS

# Construct a new dataframe with the 10 closest neighbours (most similar)
# for each boardgame.
data_neighbours = pd.DataFrame(index=data_matrix.columns, columns=range(1,11))
for i in list(range(0, len(data_matrix.columns))):
    data_neighbours.iloc[i,:10] = data_matrix.iloc[0:,i].sort_values(ascending=False)[:10].index

category = 'Adventure'
category_index = data[data.Name == category].index.tolist()[0]

# Get the boardgames in selected category.
category_boardgames = data_items.iloc[category_index]
category_boardgames = category_boardgames[category_boardgames >0].index.values

# Construct the neighbourhood from the most similar items to the
# boardgames in selected category.
most_similar_to_likes = data_neighbours.iloc[category_boardgames]
similar_list = most_similar_to_likes.values.tolist()
similar_list = list(set([item for sublist in similar_list for item in sublist]))
neighbourhood = data_matrix[similar_list].iloc[similar_list]

# A category vector containing only the neighbourhood items and
# the known boardgames.
category_vector = data_items.iloc[category_index].iloc[similar_list]

# Calculate the score.
score = neighbourhood.dot(category_vector).div(neighbourhood.sum(axis=1))

# Drop the known boardgames.
score = score.drop(category_boardgames)

print(category_boardgames)
print(score.nlargest(20))

ValueError: invalid literal for int() with base 10: 'Gloomhaven'